In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from modules.verify_names import combined_similarity, word_overlap_similarity, lexical_similarity
import os
import re
import numpy as np
from typing import List
# Sem limite de colunas
pd.set_option('display.max_columns', None)

In [2]:
mapeamento = {
    'Fabricante': 'fabricante',
    'Marca': 'marca',
    'SKU': 'sku',
    'N° modelo': 'no_modelo',
    'modelo': 'no_modelo',
    'Nome comercial': 'nome_comercial',
    'Capacidade': 'capacidade_armazenamento',
    'capacidade': 'capacidade_armazenamento',
    'Cor': 'cor',
    'Produto': 'nome_produto',
    'Empresa': 'empresa',
    'Seller': 'vendedor',
    'Frete mínimo': 'frete',
    'frete_minimo': 'frete',
    'Estoque': 'estoque',
    'Link': 'url',
    'Preço pix': 'preco_pix',
    'Preço boleto': 'preco_boleto',
    'Preço cartão 1x': 'preco_x1_cartao',
    'Preço cartão ~x s/ juros': 'preco_prazo_sem_juros_cartao_normal',
    'Quantidade parcelas s/ juros': 'qtd_parcelas_sem_juros_cartao_normal',
    'Preço cartão ~x c/ juros': 'preco_prazo_com_juros_cartao_normal',
    'Quantidade parcelas c/ juros': 'qtd_parcelas_com_juros_cartao_normal',
    'Taxa de juros': 'taxa_juros_cartao_normal',
    'Preço cartão próprio 1x': 'preco_x1_cartao_proprio',
    'Preço cartão próprio ~x s/ juros': 'preco_prazo_sem_juros_cartao_proprio',
    'Quantidade parcelas s/ juros cartão próprio': 'qtd_parcelas_sem_juros_cartao_proprio',
    'Preço cartão próprio ~x c/ juros': 'preco_prazo_com_juros_cartao_proprio',
    'Quantidade parcelas c/ juros cartão próprio': 'qtd_parcelas_com_juros_cartao_proprio',
    'Taxa de juros cartão próprio': 'taxa_juros_cartao_proprio',
    # Adicionamos 'ean', 'cep', 'data_extracao' que não estavam no mapeamento anterior
    'ean': 'ean',
    'cep': 'cep',
    'Data da extração': 'data_extracao',
    'data_extração': 'data_extracao'
}

# Função para adicionar colunas vazias necessárias ao dataframe
def adicionar_colunas_vazias(df, colunas_necessarias):
    for coluna in colunas_necessarias:
        if coluna not in df.columns:
            df[coluna] = pd.NA  # Cria a coluna com valores vazios
    return df

def extract_capacity(name):
    matches = re.findall(r"(\d+)\s*(gb|tb|ssd|hd|hdd)", name, re.IGNORECASE)
    max_capacity = 0
    for match in matches:
        capacity, unit = match
        capacity = int(capacity)
        if unit.lower() == "tb":
            capacity *= 1024  # Convertendo TB para GB
        max_capacity = max(max_capacity, capacity)

    if max_capacity > 0:
        if max_capacity < 1024:
            return f"{max_capacity}GB"
        else:
            return f"{int(max_capacity/1024)}TB"
    else:
        return None

In [3]:
# Leitura da lista de devices
path_devices = r'C:\Users\MattioliMichael(Bip)\BUSINESS INTEGRATION PARTNERS SPA\Pricing Vivo - General\4.0 Conteudos de projeto\Requisitos\lista devices.xlsx'

# Captura a aba 'devices'
devices = pd.read_excel(path_devices, sheet_name='devices')
devices = devices.drop_duplicates()
devices

,CATEGORIA,VERTICAL,SUBCATEGORIA_GERENCIAL,NOME_COMERCIAL,Modelo,Modelo com cor,Cor,COD_SAP,ID_DPGC
0,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,"Adaptador Lightning Apple 3,5mm","ADAPTADOR LIGTHNING APPLE 3,5MM","Adaptador Lightning Apple 3,5mm Branco",Branco,22018331,APL06
1,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,Adaptador Usb-C Para Conector De Fones De Ouvi...,Samsung Adaptador USB-C para P2,Adaptador Usb-C Para Conector De Fones De Ouvi...,Preto,22019829,SAM472
2,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,Cabo De Usb-C Para Lightning (1 M),Cabo USB-C Lightning Apple,Cabo De Usb-C Para Lightning (1 M) Branco,Branco,22020345,APP154
3,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,Cabo Lightning Apple 1m,CABO LIGTHNING APPLE 1M,Cabo Lightning Apple 1m Branco,Branco,22020007,APL01
4,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,"Cabo Usb Para Lightning 1,2m","I2GO CABO USB PARA LIGHTNING 1,2M","Cabo Usb Para Lightning 1,2m Branco",Branco,22019883,SAM1
...,...,...,...,...,...,...,...,...,...
828,SMARTPHONE,SMARTPHONE,SMARTPHONE,Samsung Galaxy Z Fold 5 1tb,Samsung Galaxy F946B 1TB,Samsung Galaxy Z Fold 5 1tb Preto,Preto,TGSA49062000,SAM580
829,SMARTPHONE,SMARTPHONE,SMARTPHONE,Samsung Galaxy Z Fold 5 512gb,Samsung Galaxy F946B 512GB,Samsung Galaxy Z Fold 5 512gb Preto,Preto,TGSA48962000,SAM560
830,SMARTPHONE,SMARTPHONE,SMARTPHONE,Semp Tcl Go3c Plus,SEMP TCL GO3c Plus,Semp Tcl Go3c Plus Preto,Preto,TGTL00662000,ALC156
831,SMARTPHONE,SMARTPHONE,SMARTPHONE,Semp Tcl Go5c,Alcatel 5033E,Semp Tcl Go5c Preto,Preto,TGTL00362000,ALC147


In [4]:
# Leitura das planilhas exceis para juntar as bases
path_data = r'C:\Users\MattioliMichael(Bip)\BUSINESS INTEGRATION PARTNERS SPA\Pricing Vivo - General\4.0 Conteudos de projeto\Extrações'

# Leitura de cada excel do path_data e concatenação em um único dataframe
# Utilize as bibliotecas os e sys para leitura do diretorio

# Lista de colunas necessárias
colunas_necessarias = [
    'fabricante', 'marca', 'ean', 'sku', 'no_modelo', 'nome_comercial',
    'capacidade_armazenamento', 'cor', 'nome_produto', 'empresa',
    'vendedor', 'preco_pix', 'preco_boleto', 'preco_x1_cartao',
    'preco_prazo_sem_juros_cartao_normal', 'qtd_parcelas_sem_juros_cartao_normal',
    'preco_prazo_com_juros_cartao_normal', 'qtd_parcelas_com_juros_cartao_normal',
    'taxa_juros_cartao_normal', 'preco_x1_cartao_proprio',
    'preco_prazo_sem_juros_cartao_proprio', 'qtd_parcelas_sem_juros_cartao_proprio',
    'preco_prazo_com_juros_cartao_proprio', 'qtd_parcelas_com_juros_cartao_proprio',
    'taxa_juros_cartao_proprio', 'frete', 'cep', 'estoque', 'url', 'data_extracao'
]

# Lista de arquivos no diretório
lista_arquivos = os.listdir(path_data)

# Lista de arquivos excel
lista_arquivos_excel = [arquivo for arquivo in lista_arquivos if arquivo.endswith('.xlsx')]

# Lista de dataframes
lista_df = []
for arquivo in tqdm(lista_arquivos_excel):
    print(arquivo)
    df = pd.read_excel(os.path.join(path_data, arquivo))
    df = df.rename(columns=mapeamento)
    display(df.columns)
    df = adicionar_colunas_vazias(df, colunas_necessarias)
    
    lista_df.append(df[colunas_necessarias])

  0%|          | 0/15 [00:00<?, ?it/s]

Extrações Amazon.xlsx


Index(['fabricante', 'marca', 'sku', 'no_modelo', 'nome_comercial',
       'capacidade_armazenamento', 'cor', 'nome_produto', 'empresa',
       'vendedor', 'preco_pix', 'preco_boleto', 'preco_x1_cartao',
       'preco_prazo_sem_juros_cartao_normal',
       'qtd_parcelas_sem_juros_cartao_normal',
       'preco_prazo_com_juros_cartao_normal',
       'qtd_parcelas_com_juros_cartao_normal', 'taxa_juros_cartao_normal',
       'preco_x1_cartao_proprio', 'preco_prazo_sem_juros_cartao_proprio',
       'qtd_parcelas_sem_juros_cartao_proprio',
       'preco_prazo_com_juros_cartao_proprio',
       'qtd_parcelas_com_juros_cartao_proprio', 'taxa_juros_cartao_proprio',
       'frete', 'estoque', 'url'],
      dtype='object')

Extrações Americanas.xlsx


Index(['sku', 'vendedor', 'product_name', 'preco_pix', 'preco_boleto',
       'preco_x1_cartao', 'preco_prazo_sem_juros_cartao_normal',
       'qtd_parcelas_sem_juros_cartao_normal',
       'preco_prazo_com_juros_cartao_normal',
       'qtd_parcelas_com_juros_cartao_normal', 'taxa_juros_cartao_normal',
       'preco_x1_cartao_proprio', 'preco_prazo_sem_juros_cartao_proprio',
       'qtd_parcelas_sem_juros_cartao_proprio',
       'preco_prazo_com_juros_cartao_proprio',
       'qtd_parcelas_com_juros_cartao_proprio', 'taxa_juros_cartao_proprio',
       'frete', 'cep', 'estoque', 'url', 'data_extracao', 'marca',
       'fabricante', 'cor', 'ean', 'no_modelo', 'capacidade_armazenamento'],
      dtype='object')

...

Index(['fabricante', 'marca', 'ean', 'sku', 'no_modelo', 'nome_comercial',
       'capacidade_armazenamento', 'cor', 'nome_produto', 'empresa',
       'vendedor', 'preco_pix', 'preco_boleto', 'preco_x1_cartao',
       'preco_prazo_sem_juros_cartao_normal',
       'qtd_parcelas_sem_juros_cartao_normal',
       'preco_prazo_com_juros_cartao_normal',
       'qtd_parcelas_com_juros_cartao_normal', 'taxa_juros_cartao_normal',
       'preco_x1_cartao_proprio', 'preco_prazo_sem_juros_cartao_proprio',
       'qtd_parcelas_sem_juros_cartao_proprio',
       'preco_prazo_com_juros_cartao_proprio',
       'qtd_parcelas_com_juros_cartao_proprio', 'taxa_juros_cartao_proprio',
       'frete', 'cep', 'estoque', 'url', 'data_extracao', 'link'],
      dtype='object')

Extrações Tim.xlsx


Index(['fabricante', 'marca', 'ean', 'sku', 'no_modelo', 'nome_comercial',
       'capacidade_armazenamento', 'cor', 'empresa', 'nome_produto',
       'empresa.1', 'vendedor', 'preco_pix', 'preco_boleto',
       'preco_credito_1x', 'preco_prazo_sem_juros_cartao_normal',
       'qtd_parcelas_sem_juros_cartao_normal',
       'preco_prazo_com_juros_cartao_normal',
       'qtd_parcelas_com_juros_cartao_normal', 'taxa_juros_cartao_normal',
       'preco_x1_cartao_proprio', 'preco_prazo_sem_juros_cartao_proprio',
       'qtd_parcelas_sem_juros_cartao_proprio',
       'preco_prazo_com_juros_cartao_proprio',
       'qtd_parcelas_com_juros_cartao_proprio', 'taxa_juros_cartao_proprio',
       'estoque', 'url', 'data_extracao'],
      dtype='object')

Extrações Vivo.xlsx


Index(['fabricante', 'marca', 'ean', 'sku', 'no_modelo', 'nome_comercial',
       'capacidade_armazenamento', 'cor', 'nome_produto', 'empresa',
       'vendedor', 'preco_pix', 'preco_boleto', 'preco_credito_1x',
       'preco_prazo_sem_juros_cartao_normal',
       'qtd_parcelas_sem_juros_cartao_normal',
       'preco_prazo_com_juros_cartao_normal',
       'qtd_parcelas_com_juros_cartao_normal', 'taxa_juros_cartao_normal',
       'preco_x1_cartao_proprio', 'preco_prazo_sem_juros_cartao_proprio',
       'qtd_parcelas_sem_juros_cartao_proprio',
       'preco_prazo_com_juros_cartao_proprio',
       'qtd_parcelas_com_juros_cartao_proprio', 'taxa_juros_cartao_proprio',
       'estoque', 'url', 'data_extracao'],
      dtype='object')

In [5]:
df_consolidado = pd.concat(lista_df)
df_consolidado.loc[df_consolidado['capacidade_armazenamento'].isnull(), 'capacidade_armazenamento'] = df_consolidado.loc[df_consolidado['capacidade_armazenamento'].isnull(), 'nome_produto'].astype(str).apply(extract_capacity)
df_consolidado['capacidade_armazenamento'] = df_consolidado['capacidade_armazenamento'].astype(str).apply(extract_capacity)
df_consolidado.head()

,fabricante,marca,ean,sku,no_modelo,nome_comercial,capacidade_armazenamento,cor,nome_produto,empresa,vendedor,preco_pix,preco_boleto,preco_x1_cartao,preco_prazo_sem_juros_cartao_normal,qtd_parcelas_sem_juros_cartao_normal,preco_prazo_com_juros_cartao_normal,qtd_parcelas_com_juros_cartao_normal,taxa_juros_cartao_normal,preco_x1_cartao_proprio,preco_prazo_sem_juros_cartao_proprio,qtd_parcelas_sem_juros_cartao_proprio,preco_prazo_com_juros_cartao_proprio,qtd_parcelas_com_juros_cartao_proprio,taxa_juros_cartao_proprio,frete,cep,estoque,url,data_extracao
0,NaN,NaN,NaN,NaN,NaN,NaN,1TB,NaN,Apple iPhone 15 Pro Max (1 TB) — Titânio natural,Amazon,Loja iPlace,NaN,NaN,13999.00,13999.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,https://www.amazon.com.br/dp/B0CHXB4S3F,NaN
1,NaN,samsung,NaN,sm-a145mzsgzto,NaN,NaN,128GB,prata,"Samsung Galaxy A14 4G , Prata , 128GB",Amazon,Amazon.com.br,849.0,849.0,893.68,893.68,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,https://www.amazon.com.br/dp/B0BVMXPN7Q,NaN
2,NaN,samsung,NaN,m54 5g,NaN,NaN,256GB,azul marinho,Smartphone Samsung Galaxy M54 5G 256GB Azul,Amazon,Samsung Loja Oficial,NaN,1999.0,1999.00,1999.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,https://www.amazon.com.br/dp/B0C55S8287,NaN
3,NaN,nvidia,NaN,82cgs00200,NaN,NaN,512GB,blue,Notebook Lenovo ideapad Gaming 3i i7-10750H 8G...,Amazon,MOBCOMSTORE,NaN,4999.0,4999.00,4999.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,https://www.amazon.com.br/dp/B0945KCDZX,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,1TB,NaN,Apple iPhone 15 Pro Max (1 TB) — Titânio natural,Amazon,Loja iPlace,NaN,NaN,13999.00,13999.00,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,https://www.amazon.com.br/dp/B0CHXB4S3F,NaN


In [6]:
df_devices_vivo = devices[['CATEGORIA', 'VERTICAL', 'SUBCATEGORIA_GERENCIAL', 'COD_SAP','ID_DPGC','NOME_COMERCIAL','Cor']].drop_duplicates().reset_index(drop=True)
df_devices_vivo['capacidade_armazenamento'] = df_devices_vivo['NOME_COMERCIAL'].apply(extract_capacity)
df_devices_vivo

,CATEGORIA,VERTICAL,SUBCATEGORIA_GERENCIAL,COD_SAP,ID_DPGC,NOME_COMERCIAL,Cor,capacidade_armazenamento
0,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,22018331,APL06,"Adaptador Lightning Apple 3,5mm",Branco,None
1,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,22019829,SAM472,Adaptador Usb-C Para Conector De Fones De Ouvi...,Preto,None
2,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,22020345,APP154,Cabo De Usb-C Para Lightning (1 M),Branco,None
3,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,22020007,APL01,Cabo Lightning Apple 1m,Branco,None
4,ESSENCIAIS,OVVI E ESSENCIAIS,CABOS,22019883,SAM1,"Cabo Usb Para Lightning 1,2m",Branco,None
...,...,...,...,...,...,...,...,...
792,SMARTPHONE,SMARTPHONE,SMARTPHONE,TGSA49062000,SAM580,Samsung Galaxy Z Fold 5 1tb,Preto,1TB
793,SMARTPHONE,SMARTPHONE,SMARTPHONE,TGSA48962000,SAM560,Samsung Galaxy Z Fold 5 512gb,Preto,512GB
794,SMARTPHONE,SMARTPHONE,SMARTPHONE,TGTL00662000,ALC156,Semp Tcl Go3c Plus,Preto,None
795,SMARTPHONE,SMARTPHONE,SMARTPHONE,TGTL00362000,ALC147,Semp Tcl Go5c,Preto,None


In [29]:
list_cruzamento_devices = []
for _, row in tqdm(df_devices_vivo.iterrows(), total=df_devices_vivo.shape[0]):
    categoria = row['CATEGORIA']
    vertical = row['VERTICAL']
    subcategoria_gerencial = row['SUBCATEGORIA_GERENCIAL']
    cod_sap = row['COD_SAP']
    id_dpgc = row['ID_DPGC']
    nome_comercial = row['NOME_COMERCIAL']
    cor = row['Cor']
    
    dict_cruzamento_devices = {
        'categoria': categoria,
        'vertical': vertical,
        'subcategoria_gerencial': subcategoria_gerencial,
        'cod_sap': cod_sap,
        'id_dpgc': id_dpgc,
        'nome_produto': nome_comercial,
        'cor': cor
    }

    
    df_consolidado['name_similarity_overlap'] = df_consolidado['nome_produto'].astype(str).apply(lambda x: word_overlap_similarity(x, nome_comercial))
    df_consolidado['name_similarity_lexical'] = df_consolidado['nome_produto'].astype(str).apply(lambda x: lexical_similarity(x, nome_comercial))
    df_consolidado['mean_overlap_lexical_similarity'] = df_consolidado[['name_similarity_overlap', 'name_similarity_lexical']].mean(axis=1)

    df_consolidado_filtrado = df_consolidado[df_consolidado['mean_overlap_lexical_similarity'] > 0.35].reset_index(drop=True).copy()
    df_consolidado_filtrado['similarity'] = df_consolidado_filtrado['nome_produto'].astype(str).progress_apply(lambda x: combined_similarity(x, nome_comercial, 0.7, 0.3))

    df_consolidado_filtrado = df_consolidado_filtrado[df_consolidado_filtrado['similarity'] > 0.6].reset_index(drop=True)
    df_consolidado_filtrado['CATEGORIA'] = categoria
    df_consolidado_filtrado['VERTICAL'] = vertical
    df_consolidado_filtrado['SUBCATEGORIA_GERENCIAL'] = subcategoria_gerencial
    df_consolidado_filtrado['COD_SAP'] = cod_sap
    df_consolidado_filtrado['ID_DPGC'] = id_dpgc
    df_consolidado_filtrado['NOME_COMERCIAL'] = nome_comercial
    df_consolidado_filtrado['Cor'] = cor
    list_cruzamento_devices.append(df_consolidado_filtrado)
    


  0%|          | 0/797 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/77 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [31]:
df_cruzamento_devices = pd.concat(list_cruzamento_devices)
df_cruzamento_devices.to_excel('cruzamento_devices.xlsx', index=False)